# Quick introduction

erddapy is a pure python package and can be installed with conda

```shell
conda install --channel conda-forge erddapy
```

or pip

```shell
pip install erddapy
```


First we need to instantiate the ERDDAP URL constructor for a server.
In this example we will use
[https://erddap.ifremer.fr/erddap](https://erddap.ifremer.fr/erddap/index.html).

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server="ifremer",
    protocol="tabledap",
    response="csv",
)

Now we can populate the object a dataset id, variables of interest, and its
constraints (last week gliders). Use the method `to_pandas` to download the
csv(p) response, a comma separated values with units and explore the Dataframe.


In [ ]:
e.dataset_id = "OceanGlidersGDACTrajectories"

e.variables = [
    "time",
    "latitude",
    "longitude",
    "PRES",
    "PSAL",
    "TEMP",
]

e.constraints = {
    "time>=": "2000-01-01",
}


df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
).dropna()

df.head()

We can constraint the in time and space with relative constraints like in the
example below. For more ways to access the data please check the "Longer
introduction."


In [ ]:
constraints = {
    "time>=": "now-14days",
    "time<=": "now-7days",
    "longitude<=": "min(longitude)+10",
    "longitude>=": "min(longitude)",
}


url = e.get_download_url(
    response="htmlTable",
    constraints=constraints,
)

print(url)

The same request as above but with non-relative constraints. Note that these
values will change if we run this at a later time.


In [ ]:
constraints = {
    "time>=": "2023-02-06T19:12:32Z",
    "time<=": "2023-02-13T19:12:32Z",
    "longitude>=": -149.60,
    "longitude<=": -139.60,
}

url = e.get_download_url(
    response="html",
    constraints=constraints,
)

print(url)

We can search all datasets with a set of constraints by setting `dataset_id` to `"allDatasets"`. Note that these variables are different than the ones available at the individual dataset level. For a reference of the possible variables to query all datasets see the `<server-url>/erddap/<protocol>/allDatasets.html` page, like [this one](https://erddap.ifremer.fr/erddap/tabledap/allDatasets.html) for the Ifremer ERDDAP server.

In [ ]:
e.dataset_id = "allDatasets"

e.variables = [
    "datasetID",
    "institution",
    "minTime",
    "maxTime",
    "minLatitude",
    "maxLatitude",
    "minLongitude",
    "maxLongitude",
]


e.constraints = {
    "minTime>=": "1997-01-01T00:00:00Z",
}


url = e.get_download_url(response="html")
print(url)

df = e.to_pandas(
    index_col="datasetID",
    parse_dates=True,
).dropna()


df.head()